In [142]:
import pandas as pd
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVR

In [156]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [157]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)
df = df.dropna()

In [173]:
df.head(5)

,REST_ID,TABLE SIZE,TIME (SEC),MONTH,DAY,TIME CHECKIN,WAIT TIME
0,113442,5,40159,1,5,40601,442
1,113442,4,40545,1,5,40600,55
2,113442,2,40631,1,5,40641,10
3,113442,2,41681,1,5,41693,12
4,113442,5,42660,1,5,43236,576


#### Parsing the DateTime and adding columns based on it

In [167]:
dater = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S.%f")
dater2 = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S")

In [170]:
df["TIME CHECKIN"] = df["CHECKINTIME"].apply(lambda a: dater2(a).hour*3600 + dater2(a).minute*60 + dater2(a).second)
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: dater(a).hour*3600 + dater(a).minute*60 + dater(a).second)
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[0].split('-')[1]))
df["DAY"] = df["CREATE TIME"].apply(lambda a: dater(a).isoweekday())
df["WAIT TIME"] = df["TIME CHECKIN"] - df["TIME (SEC)"]

In [172]:
df.drop("CREATE TIME",1, inplace = True)
df.drop("CHECKINTIME",1, inplace = True)

#### Setting up the regression model and training on it

In [ ]:
clf = SVR()
X = 